In [ ]:
#Separa video em frames, identifica os pontos do rosto, pega a abertura dos olhos e da boca.

In [ ]:
#retona distancia entre dois pontos
def distancia(a,b):
    return math.sqrt(pow(b.x-a.x,2) + pow(b.y-a.y,2))
#retorna ear do olho, recebe todos os ponto, e o ponto inical do olho
def ear(pontos,inicial):
    A = distancia(pontos.part(inicial+1),pontos.part(inicial+5)) + distancia(pontos.part(inicial+2),pontos.part(inicial+4))
    C = distancia(pontos.part(inicial+0),pontos.part(inicial+3))
    return A/(2*C)

In [ ]:
#Importa bibliotecas
from scipy.spatial import distance as dist
import cv2
import numpy as np
import os
import dlib
import pandas as pd
import math

#Abre o video e configura
cap = cv2.VideoCapture('../Videos/example_.mp4') #Carrega Video
cap.set(cv2.CAP_PROP_FPS, 30) #Seta o FPS para 30

#Inicia variaveis e dependencias
detector = dlib.get_frontal_face_detector() #Mode de predição de rosto
predictor = dlib.shape_predictor("../Dependencias/shape_predictor_68_face_landmarks.dat") #Modelo de predição dos postos do rosto
currentFrame = 0

#Cria diretorio para salvear iamgens
try:
    if not os.path.exists('../data'):
        os.makedirs('../data')
except OSError:
    print ('Error: Creating directory of data')

    
#Percorre os frames e identifica rosto e seus pontos, pega a abertura dos olhos e da boca, escreve na imagem e salva.

while(True):
    
    ret, frame = cap.read()
    currentFrame += 1
    
    if ret:
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #passa o frame para preto e branco
        detections = detector(gray, 1) #detecta as faces na imagem e retorna um retangulo de sua posição
        
        if detections: #se foi identificado algun rosto no frame
            
            for k,rosto in enumerate(detections): #pegamos os cada rosto identificado
                shape = predictor(gray, rosto) #mandamos o rosto e recebemos os pontos do mesmo.
                
                #Liga os pontos do meio da Boca na imagem
                cv2.line(frame, (shape.part(62).x, shape.part(62).y), (shape.part(66).x, shape.part(66).y), (0,255,0), 1)
                #Calcul e escreve na imagem as distancias dos olhos e da boca
                cv2.putText(frame, "Olho d: " + str(ear(shape,36)), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                cv2.putText(frame, "Olho e: " + str(ear(shape,42)), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                cv2.putText(frame, "Boca  : " + str(distancia(shape.part(62),shape.part(66))), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        
        name = '../data/frame' + str(currentFrame) + '.jpg'
        #print ('Creating...' + name)
        cv2.imwrite(name, frame)
        
    else:
        break


cap.release()
cv2.destroyAllWindows()